# Assignment 1

### Question 0: Data Preparation

In [13]:
! pip install pyreadstat

In [14]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm
import seaborn as sns

In [15]:
df = pd.read_csv("data.csv", sep=';')
df.tail()
df.head()

,Review_ID,Number_Words_Review,ZNumber_Words_Review,Prod_Desig,Prod_Design_positive,Prod_Design_negative,Prod_Technical,Prod_Technical_positive,Prod_Technical_negative,Prod_Price,...,Serv_Delivery_positive,Serv_Delivery_negative,Category,Country,Gender,Age,Sentiment,Rating_Score,Purchase,Number_of_Purchases
0,1,34,"-0,178828858",1,1,0,1,1,0,0,...,0,0,0,1,0,"24,56327497",4,5,1,1
1,2,32,"-0,225796385",1,1,0,1,1,0,1,...,0,0,0,1,0,"38,27120469",5,4,0,0
2,3,54,"0,290846409",1,0,1,1,0,1,1,...,0,1,0,1,0,"46,77239881",2,2,0,0
3,4,20,"-0,507601545",1,1,0,1,1,0,0,...,0,0,0,1,0,"31,66539161",5,5,1,2
4,5,12,"-0,695471652",1,1,0,1,1,0,0,...,0,0,0,1,0,"25,31939178",3,5,0,0


### Convert 'object' columns to 'float'

In [16]:
df['ZNumber_Words_Review'] = df['ZNumber_Words_Review'].astype(str).str.replace(',', '.').astype(float)
df['Age'] = df['Age'].astype(str).str.replace(',', '.').astype(float)
# Check if the conversion was successful
df[['ZNumber_Words_Review', 'Age']].info(), df[['ZNumber_Words_Review', 'Age']].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ZNumber_Words_Review  600 non-null    float64
 1   Age                   600 non-null    float64
dtypes: float64(2)
memory usage: 9.5 KB


(None,
    ZNumber_Words_Review        Age
 0             -0.178829  24.563275
 1             -0.225796  38.271205
 2              0.290846  46.772399
 3             -0.507602  31.665392
 4             -0.695472  25.319392)

### Fix 'Category' column

In [17]:
category_dummies = pd.get_dummies(df['Category'], prefix='Category')
category_dummies = category_dummies.astype(int)
df = df.drop('Category', axis=1)
df = df.join(category_dummies)
df.head()

,Review_ID,Number_Words_Review,ZNumber_Words_Review,Prod_Desig,Prod_Design_positive,Prod_Design_negative,Prod_Technical,Prod_Technical_positive,Prod_Technical_negative,Prod_Price,...,Country,Gender,Age,Sentiment,Rating_Score,Purchase,Number_of_Purchases,Category_0,Category_1,Category_2
0,1,34,-0.178829,1,1,0,1,1,0,0,...,1,0,24.563275,4,5,1,1,1,0,0
1,2,32,-0.225796,1,1,0,1,1,0,1,...,1,0,38.271205,5,4,0,0,1,0,0
2,3,54,0.290846,1,0,1,1,0,1,1,...,1,0,46.772399,2,2,0,0,1,0,0
3,4,20,-0.507602,1,1,0,1,1,0,0,...,1,0,31.665392,5,5,1,2,1,0,0
4,5,12,-0.695472,1,1,0,1,1,0,0,...,1,0,25.319392,3,5,0,0,1,0,0


## Question 1: Linear Regression

Given the nature of the question, a regression model seems appropriate as we are interested in understanding how various factors influence the continuous outcome of rating scores.

In [19]:
target_reg = 'Rating_Score'
X_reg = df.drop(['Review_ID', target_reg], axis=1)
y_reg = df[target_reg]

For this analysis, I will build a linear regression model to predict Rating_Score using the explanatory variables provided. This model will help us understand which factors are significant in explaining the variation in rating scores and the direction of their impact.

In [20]:
X_reg = sm.add_constant(X_reg)
model = sm.OLS(y_reg, X_reg).fit()
predictions = model.predict(X_reg)
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:           Rating_Score   R-squared:                       0.706
Model:                            OLS   Adj. R-squared:                  0.696
Method:                 Least Squares   F-statistic:                     69.56
Date:                Wed, 20 Mar 2024   Prob (F-statistic):          2.19e-139
Time:                        11:26:10   Log-Likelihood:                -612.33
No. Observations:                 600   AIC:                             1267.
Df Residuals:                     579   BIC:                             1359.
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                    1

Results: 
- Sentiment Score: This has a significant positive impact on rating scores. A higher sentiment score is associated with higher rating scores.
- Product Technical Features: Mentioning technical features (both positively and negatively) has a significant effect. In particular, positive mentions of technical features strongly increase rating scores, while the mere mention of technical features (regardless of sentiment) seems to decrease them.
- Service Delivery: Negative mentions of delivery have a significant negative impact on rating scores, whereas positive mentions have a positive impact.

Marketing Implications:

- Focusing on improving customer sentiment about products can lead to higher rating scores. Strategies to enhance positive experiences and minimize negative ones are crucial.
- Emphasizing the technical features of products in marketing and communication strategies seems beneficial, especially highlighting the positive aspects.
- Efficient and satisfactory delivery services can significantly influence customer ratings. Efforts to improve delivery experiences should be prioritized.

## Question 2: Logistic Regression

Create 'Purchase_Occurance'

In [ ]:
# Create a new column 'Purchase_Occurrence' which is 1 if 'Number_of_Purchases' is greater than 0, else 0
df['Purchase_Occurrence'] = (df['Number_of_Purchases'] > 0).astype(int)

# Display the first few rows to confirm the new column
df[['Number_of_Purchases', 'Purchase_Occurrence']].head()


,Number_of_Purchases,Purchase_Occurrence
0,1,1
1,0,0
2,0,0
3,2,1
4,0,0


In [ ]:
target_log = 'Purchase_Occurrence'
X_log = df.drop(['Review_ID', target_log], axis=1)
y_log = df[target_log]

In [22]:
X_log = sm.add_constant(X_log)
logit_mod = sm.Logit(y_log, X_log)
logit_res = logit_mod.fit()
print(logit_res.summary())

         Current function value: 0.000004
         Iterations: 35
                            Logit Regression Results                           
Dep. Variable:     Purchase_Occurrence   No. Observations:                  600
Model:                           Logit   Df Residuals:                      578
Method:                            MLE   Df Model:                           21
Date:                 Wed, 20 Mar 2024   Pseudo R-squ.:                   1.000
Time:                         11:47:24   Log-Likelihood:             -0.0021885
converged:                       False   LL-Null:                       -385.28
Covariance Type:             nonrobust   LLR p-value:                1.574e-149
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      -9.9499        nan        nan        nan         nan         nan
Number_Words_Revie

d:\Anaconda\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Results:

- Sentiment Score: A significant positive predictor, indicating that higher sentiment scores increase the likelihood of a purchase.
- Product Design: Mentioning design in reviews significantly increases the likelihood of purchase, though the positive and negative aspects of design are not significant predictors.
- Product Technical Features: No significant effect was found, despite the presence of these variables in the model.
- Service Delivery: This variable and its subcategories have problematic estimates (possibly due to collinearity or data issues).
- Category and Country: These demographic factors significantly affect purchase likelihood.

Marketing Implications:

- Enhancing customer sentiment through quality products and services can increase the likelihood of purchase.
- Emphasis on design in marketing and product development could be beneficial, as design mentions in reviews correlate with purchase decisions.
- Attention to demographic factors like country and product category can help tailor marketing strategies effectively.

## Question 3

## Question 4